
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>

<i18n value="263caa08-bb08-4022-8d8f-bd2f51d77752"/>

# 分類：(Classification:)ロジスティック回帰

ここまでは、回帰のユースケースのみを検証してきました。では、分類の扱い方を見てみましょう。

このラボでは、同じAirbnbのデータセットを使用しますが、価格を予測する代わりに、ホストが<a href="https://www.airbnb.com/superhost" target="_blank">スーパーホスト</a>であるかどうかを予測します。

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) このレッスンでは以下を行います。<br>
 - ロジスティック回帰モデルを構築
 - モデルの性能を評価するための様々なメトリックスを使用

In [0]:
%run "../Includes/Classroom-Setup"

In [0]:
file_path = f"{DA.paths.datasets}/airbnb/sf-listings/sf-listings-2019-03-06-clean.delta/"
airbnb_df = spark.read.format("delta").load(file_path)

<i18n value="3f07e772-c15d-46e4-8acd-866b661fbb9b"/>

## ベースラインモデル (Baseline Model)

機械学習モデルを構築する前に、比較するためのベースラインモデルを構築します。まず、ホストが <a href="https://www.airbnb.com/superhost" target="_blank">superhost</a> であるかどうかを予測することから始めます。

ベースラインモデルでは、誰もスーパーホストではないことを予測し、その精度を評価することにしています。他の指標については、後ほどより複雑なモデルを構築する際に検討する予定です。

0. **`host_is_superhost`** カラム (t/f) を 1/0 に変換し、結果のカラムを **`label`** と呼びます。その後、 **`host_is_superhost`** を DROPします。
0. 結果のDataFrameに、全ての値を **`0.0`** にした **`prediction`** カラムを追加します。誰もスーパーホストではないことを常に予測するようにします。

この2つのステップを終えたら、次に「モデル」の精度を評価します。

便利な機能がいくつかあります。
* <a href="https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.when.html" target="_blank">when()</a>
* <a href="https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.withColumn.html" target="_blank">withColumn()</a>
* <a href="https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.lit.html" target="_blank">lit()</a>

In [0]:
# ANSWER

from pyspark.sql.functions import when, col, lit

label_df = airbnb_df.select(when(col("host_is_superhost") == "t", 1.0).otherwise(0.0).alias("label"), "*").drop("host_is_superhost")

pred_df = label_df.withColumn("prediction", lit(0.0))

<i18n value="d04eb817-2010-4021-a898-42ca8abaa00d"/>

## モデル評価

とりあえず、「accuracy」を指標にしましょう。<a href="https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.evaluation.MulticlassClassificationEvaluator.html?highlight=multiclassclassificationevaluator#pyspark.ml.evaluation.MulticlassClassificationEvaluator" target="_blank">MulticlassClassificationEvaluator</a> を使います。

In [0]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

mc_evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
print(f"The accuracy is {100*mc_evaluator.evaluate(pred_df):.2f}%")

<i18n value="5fe00f31-d186-4ab8-b6bb-437f7ddc4a00"/>

## Train-Test分割 (Train-Test Split)

よしっ!これでベースラインモデルができました。次に、データを学習用とテスト用に分割します。

In [0]:
train_df, test_df = label_df.randomSplit([.8, .2], seed=42)
print(train_df.cache().count())

<i18n value="a7998d44-af91-4dfa-b80c-8b96ebfe5311"/>

## 可視化 (Visualize)

トレーニングデータセットにおける **`review_scores_rating`** と **`label`** の関係性を見てみましょう。

In [0]:
display(train_df.select("review_scores_rating", "label"))

<i18n value="1ce4ba05-f558-484d-a8e8-53bde1e119fc"/>

## ロジスティック回帰 (Logistic Regression)

ここで、すべての特徴量を使用して、<a href="https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.classification.LogisticRegression.html?highlight=logisticregression#pyspark.ml.classification.LogisticRegression" target="_blank">ロジスティック回帰モデル</a>を構築します（ヒント：RFormulaを使用します）。前処理ステップとロジスティック回帰モデル構築ステップをPipelineに入れます。

In [0]:
# ANSWER
from pyspark.ml import Pipeline
from pyspark.ml.feature import RFormula
from pyspark.ml.classification import LogisticRegression

r_formula = RFormula(formula="label ~ .", 
                    featuresCol="features", 
                    labelCol="label", 
                    handleInvalid="skip") # Look at handleInvalid

lr = LogisticRegression(labelCol="label", featuresCol="features")
pipeline = Pipeline(stages=[r_formula, lr])
pipeline_model = pipeline.fit(train_df)
pred_df = pipeline_model.transform(test_df)

<i18n value="3a06d71c-8551-44c8-b33e-8ae40a443713"/>

## 評価 (Evaluate)

AUROCは何に役に立つでしょうか？PR曲線下の面積のような評価指標を追加します。

In [0]:
# ANSWER
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

mc_evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
print(f"The accuracy is {100*mc_evaluator.evaluate(pred_df):.2f}%")

bc_evaluator = BinaryClassificationEvaluator(metricName="areaUnderROC")
print(f"The area under the ROC curve: {bc_evaluator.evaluate(pred_df):.2f}")

bc_evaluator.setMetricName("areaUnderPR")
print(f"The area under the PR curve: {bc_evaluator.evaluate(pred_df):.2f}")

<i18n value="0ef0e2b9-6ce9-4377-8587-83b5260fd05a"/>

## ハイパーパラメータチューニングの追加 (Add Hyperparameter Tuning)

クロスバリデーターを用いて、ロジスティック回帰モデルのハイパーパラメーターを変更します。どの程度、指標を改善できますか？

In [0]:
# ANSWER
from pyspark.ml.tuning import ParamGridBuilder
from pyspark.ml.tuning import CrossValidator

param_grid = (ParamGridBuilder()
            .addGrid(lr.regParam, [0.1, 0.2])
            .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])
            .build())

cv = CrossValidator(estimator=lr, evaluator=mc_evaluator, estimatorParamMaps=param_grid,
                    numFolds=3, parallelism=4, seed=42)

pipeline = Pipeline(stages=[r_formula, cv])

pipeline_model = pipeline.fit(train_df)

pred_df = pipeline_model.transform(test_df)

<i18n value="111f2dc7-5535-45b7-82f6-ad2e5f2cbf16"/>

## 再評価 (Evaluate again)

In [0]:
mc_evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
print(f"The accuracy is {100*mc_evaluator.evaluate(pred_df):.2f}%")

bc_evaluator = BinaryClassificationEvaluator(metricName="areaUnderROC")
print(f"The area under the ROC curve: {bc_evaluator.evaluate(pred_df):.2f}")

<i18n value="7e88e044-0a34-4815-8eab-1dc37532a082"/>

## スーパーボーナス (Super Bonus)

MLflowを使って実験を記録してみましょう。

&copy; 2022 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="https://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="https://help.databricks.com/">Support</a>